## ANALISIS DE LA VERIFICACION MANUAL ##

In [ ]:
#pip install pysentimiento
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [74]:
import datetime
df_pol = pd.read_excel('POLITICO_VM.xlsx')
df_pand = pd.read_excel('PANDEMIA_VM.xlsx')
#df = df.iloc[:100,:]
#df_final.info()
df_pand.columns

Index(['id', 'tweet_sentence', 'target_PY', 'target_E1', 'I_V1', 'target_E2',
       'I_V2', 'target_E3', 'I_V3'],
      dtype='object')

In [83]:
len(df_pol)

801

*El formato de llenado fue en excel, por lo cual podrian producirse algunas imperfecciones al momomento de etiquedo manual. El siguiente pedazo de codigo genera una limpieza de aquellos espacios.*

In [66]:
def Limpiar_columna(df_, label):
    #df_[[label]] = df_[[label]].astype(str)
    #print(df_[[label]].info())
    df_[label] = df_[label].apply(lambda x: x.replace(" ", ""))
    #df_[label] = df_[label].apply(lambda x: print(type(x)))
    return df_

*Para la revision comparamos los tres etiquetas producidas por cada etiquetador a partir de las siguientes condiciones:*
1. Si de las tres etiquetas obtenidas, basta que el número de ocurrencias de una de las categorias supere 2, el target final correspondera a esa categoria.
2. Si de las tres etiquetas obtenidas, basta que el numero de ocurrencias de una de las categorıas es igual a 1, el target final es ambiguo por lo que el tweet es descartado.


In [87]:
def procesar_revision(df, name_archivo):
    #df_original = df.copy()
    #df_original = df.reset_index(drop = True)
    indice_eliminado = []
    target_final = []
    for i, row in df.iterrows():
        if row['I_V1'] +  row['I_V2'] +  row['I_V3'] > 1:
            indice_eliminado.append(i)
        else:
            sub_list = [row['target_E1'], row['target_E2'], row['target_E3']]
            conteo = Counter(sub_list)
            #print(conteo)
            if sub_list.count('POS') > 1 or sub_list.count('NEG') > 1 or sub_list.count('NEU') > 1:
                max_value = conteo.most_common()[0][0]
                target_final.append(max_value)                
            else:
                indice_eliminado.append(i)
            #print(max(conteo.most_common(1)))
            #if (row['target_E1'] and row['target_E1']) & (row['target_E2'] and row['target_E3']) & (row['target_E1'] and row['target_E3']):
        
        
    #print(len(df_original))
    #df_eliminar= df.iloc[indice_eliminado]
    df_VM = df.drop(indice_eliminado)
    print('Se han eliminado', len(indice_eliminado),' tweets por ambiguedad')
    df_VM = df_VM.reset_index(drop = True)
    df_VM['target_final'] = target_final
    
    count_cambio = 0
    for i, row in df_VM.iterrows():
         if row['target_PY'] !=  row['target_final']:
                count_cambio = count_cambio + 1
    print('Se han modificado el sentimiento de', count_cambio,' tweets mediante la verificacion manual de', len(df))
    print(len(df_VM))
    return df_VM[['id','tweet_sentence','target_final']].to_csv(name_archivo)
    #return df_revision, df_original

In [88]:
procesar_revision(df_pol,'pol_VM_final.csv')

Se han eliminado 49  tweets por ambiguedad
Se han modificado el sentimiento de 330  tweets mediante la verificacion manual de 801
752


In [97]:
df = pd.read_csv('pand_VM_final.csv', delimiter = ',')
df.target_final.value_counts()

POS    337
NEU    260
NEG    218
Name: target_final, dtype: int64

In [90]:
procesar_revision(df_pand,'pand_VM_final.csv')

Se han eliminado 91  tweets por ambiguedad
Se han modificado el sentimiento de 362  tweets mediante la verificacion manual de 906
815
